In [ ]:
pip install scapy

In [ ]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

Captura de 10 paquetes

In [ ]:
num_of_packets_to_sniff = 10
pcap = sniff(count=num_of_packets_to_sniff)

# rdpcap returns packet list
## packetlist object can be enumerated
print(type(pcap))
print(len(pcap))
print(pcap)
pcap[0]


<class 'scapy.plist.PacketList'>
10
<Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>


<Ether  dst=02:42:ac:1c:00:0c src=02:42:d6:41:2e:f7 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=505 id=28858 flags=DF frag=0 ttl=64 proto=6 chksum=0x6fff src=172.28.0.1 dst=172.28.0.12 |<TCP  sport=53014 dport=8080 seq=3825533892 ack=1062884587 dataofs=8 reserved=0 flags=PA window=501 chksum=0x5a31 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (3634023909, 1945160409))] |<Raw  load='GET /socket.io/?EIO=3&sid=2AWrOKt2nU01HZoqAAAC&t=OwSjOKH&transport=polling HTTP/1.1\r\nHost: colab.research.google.com\r\nUser-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0,gzip(gfe)\r\nAccept: */*\r\nAccept-Encoding: gzip, deflate, br,gzip(gfe)\r\nAccept-Language: en-US,en;q=0.5\r\nReferer: https://colab.research.google.com/\r\nSec-Fetch-Dest: empty\r\nSec-Fetch-Mode: cors\r\nSec-Fetch-Site: same-origin\r\nTe: trailers\r\n\r\n' |>>>>

TCP

In [ ]:
"""
0                   1                   2                   3
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|          Source Port          |       Destination Port        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                        Sequence Number                        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Acknowledgment Number                      |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|  Data |           |U|A|P|R|S|F|                               |
| Offset| Reserved  |R|C|S|S|Y|I|            Window             |
|       |           |G|K|H|T|N|N|                               |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|           Checksum            |         Urgent Pointer        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Options                    |    Padding    |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                             data                              |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
02 04 05 a0 01 03 03 05 01 01 08 0a 1d 74 65 c5 00 00 00 00 04 02 00 00
"""

print()

Exploración

In [ ]:
#Recordemos que en los modelos de red, las capas inferiores encapsulan la información de las capas superiores
#Frames en paquetes, paquetes en segmentos (Ethernet, IP), modelo TCP/IP

ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / TCP 172.28.0.1:53014 > 172.28.0.12:8080 PA / Raw
Informacion del paquete:
IP / TCP 172.28.0.1:53014 > 172.28.0.12:8080 PA / Raw
Informacion del segmento:
TCP 172.28.0.1:53014 > 172.28.0.12:8080 PA / Raw
Data:
Raw
###[ Ethernet ]### 
  dst       = 02:42:ac:1c:00:0c
  src       = 02:42:d6:41:2e:f7
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 505
     id        = 28858
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0x6fff
     src       = 172.28.0.1
     dst       = 172.28.0.12
     \options   \
###[ TCP ]### 
        sport     = 53014
        dport     = 8080
        seq       = 3825533892
        ack       = 1062884587
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 501
        chksum    = 0x5a31
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (3634023909, 19451604

Tipos de datos en Scapy

In [ ]:
print(type(ethernet_frame))
print(type(ip_packet))
print(type(segment))

# Packets can be filtered on layers ie) ethernet_frame[scapy.layers.l2.Ether]
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])

# Scapy provides this via import statements
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.TCP'>
Ethernet <Ether from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
IP <IP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
TCP <TCP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
UDP <UDP from Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>


Conversion de PCAP a DF

In [ ]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])

    field_values.append(packet.time)

    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)

    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.head(5))

  version ihl tos   len     id flags frag ttl proto chksum  ... dataofs  \
0       4   5   0   505  28858    DF    0  64     6  28671  ...       8   
1       4   5   0    52  55987    DF    0  64     6   1995  ...       8   
2       4   5   0  1239  55988    DF    0  64     6    807  ...       8   
3       4   5   0    52  28859    DF    0  64     6  29123  ...       8   
4       4   5   0   609  28860    DF    0  64     6  28565  ...       8   

  reserved flags  window chksum urgptr options payload  \
0        0    PA     501  23089      0       3     453   
1        0     A     501  22636      0       3       0   
2        0    PA     501  23823      0       3    1187   
3        0     A     492  22636      0       3       0   
4        0    PA     501  23193      0       3     557   

                                         payload_raw  \
0  b'GET /socket.io/?EIO=3&sid=2AWrOKt2nU01HZoqAA...   
1                                                b''   
2  b'HTTP/1.1 200 OK\r\nContent-

In [ ]:
# Return Src Address, Dst Address, Src Port, Dst Port
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,172.28.0.1,172.28.0.12,53014,8080
1,172.28.0.12,172.28.0.1,8080,53014
2,172.28.0.12,172.28.0.1,8080,53014
3,172.28.0.1,172.28.0.12,53014,8080
4,172.28.0.1,172.28.0.12,53014,8080
5,172.28.0.1,172.28.0.12,53014,8080
6,172.28.0.12,172.28.0.1,8080,53014
7,172.28.0.12,172.28.0.1,8080,53014
8,172.28.0.1,172.28.0.12,53014,8080
9,172.28.0.12,172.28.0.1,8080,53010


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('src').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('dst').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('sport').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('dport').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['dst'].value_counts()
    for x_label, grp in _df_4.groupby('src')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('src')
_ = plt.ylabel('dst')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['sport'].value_counts()
    for x_label, grp in _df_5.groupby('dst')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('dst')
_ = plt.ylabel('sport')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['dport'].value_counts()
    for x_label, grp in _df_6.groupby('sport')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('sport')
_ = plt.ylabel('dport')

In [ ]:
#Calcular estadisticas
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

# Top Source Address
count             10
unique             2
top       172.28.0.1
freq               5
Name: src, dtype: object 




In [ ]:
#¿Quien es la IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']
print(frequent_address)

# A que IPs les habla la IP mas frecuente
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

172.28.0.1
['172.28.0.12'] 


